In [8]:
!

In [1]:
from Bio import SeqIO # Permet l'import de la fonction parse
from Bio.SeqIO.QualityIO import FastqGeneralIterator # Permet un parse plus rapide lorsque beaucoup de séquences (fastaq uniquement sinon voir site biopython pour fasta)
import functools as ft # Permet de regrouper des listes pour radix sort
import numpy as np

import copy

from tqdm import tqdm # Permet d'estimer le temps d'éxécution sur un boucle
from datetime import datetime # Permet de comparer la vitesse de 2 programmes 

## Début du code 

In [23]:
# Bloc de test

now = datetime.now()

list_genom = genome_import()

list_reads= reads_import_cuts(10)

#list_reads2= reads_import()
"""
for el in tqdm(list_reads[0:6], desc = "Mapping") :
    position = []
    for k in tqdm(list_genom, desc = "Chromosome") :
        pos_list = string_location(k[0],el[0])
        poistion.append([k[1], pos_list])
    el.append(position)
"""
time_imp = datetime.now() - now
print("Temps total =", time_imp)
print(list_reads[0:6])

Import sequence: 1500000it [00:33, 45297.92it/s]


Temps total = 0:00:35.937630
[['TTTCCTTTTT', 'NC_004325.2-100000', 1], ['AAGCGTTTTA', 'NC_004325.2-100000', 2], ['TTTTTTAATA', 'NC_004325.2-100000', 3], ['AAAAAAATAT', 'NC_004325.2-100000', 4], ['AGTATTATAT', 'NC_004325.2-100000', 5], ['AGTAACGGGT', 'NC_004325.2-100000', 6]]


In [11]:
now_out = datetime.now()
for i in tqdm(range(len(list_genom))) :
    data = DC3(list_genom[i][0] + "$")
    list_genom[i].append(data)
    """
    filename1 = "DC3_chrom"
    filename3 = ".txt"
    filename = filename1 + str(i+1) + filename3

    export_file(data, filename, "DC3")
    """
time_imp = datetime.now() - now_out
print("Temps total =", time_imp)

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [32:55<00:00, 131.73s/it]

Temps total = 0:32:55.992169


In [11]:
# Version import :

for i in tqdm(range(len(list_genom))) :
    filename1 = "DC3_chrom"
    filename3 = ".txt"
    filename = filename1 + str(i+1) + filename3

    list_genom[i].append(import_file(filename, "DC3"))

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:11<00:00,  1.34it/s]


In [27]:
now_out = datetime.now()
bwt = BWT(list_genom[0][0],list_genom[0][2])
for i in tqdm(range(len(list_reads))) :
    list_reads[i].append(k_positioning(list_genom[0][0],list_reads[i][0], bwt,list_genom[0][2]))
time_imp = datetime.now() - now_out
print("Temps total =", time_imp)

  0%|                                                                        | 2/15000000 [00:00<1782:34:43,  2.34it/s]

No occurence of the substring was found


  0%|                                                                        | 6/15000000 [00:02<1832:14:52,  2.27it/s]

No occurence of the substring was found


  0%|                                                                        | 7/15000000 [00:03<1841:31:30,  2.26it/s]

...

No occurence of the substring was found


  0%|                                                                       | 50/15000000 [00:21<1845:15:17,  2.26it/s]

No occurence of the substring was found


  0%|                                                                       | 54/15000000 [00:24<1861:27:35,  2.24it/s]


KeyboardInterrupt: 

In [20]:
# Création des listes de mapping :

full_mapping = []
actual_mapping = []

bwt = BWT(list_genom[0][0],list_genom[0][2])
for i in tqdm(range(len(list_genom))) :
    for j in range(80) :
        actual_mapping.append([list_reads[j][2], k_positioning(list_genom[i][0],list_reads[j][0], bwt,list_genom[i][2])])
    full_mapping.append(actual_mapping)
    actual_mapping = []

print(full_mapping[0][0:10])

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [08:49<00:00, 35.28s/it]

[[1, [198275, 346984, 412675, 222197, 222208, 222219, 141536, 278529, 472382, 95770, 370439, 308410, 373485, 593890, 68418, 593802, 423536, 536793, 249822, 201291, 380365, 569951, 361808, 434544, 52645, 389576, 293273, 73765, 444636, 412568, 253527, 436364, 436171, 436187, 436203, 436219, 436235, 436251, 436267, 436283, 436299, 444994, 264242, 225936, 436348, 436315, 292693, 357755, 378191, 357263, 487228, 334026, 412447, 396565, 304229, 226151, 139442]], [2, [-1]], [3, [328588, 262315, 588735, 57953, 69246, 149784, 232280, 61180, 597598, 120607, 260329, 536798, 232049, 83477, 201685, 552580, 521235, 603304, 594254, 559547, 58555, 65241, 379377, 407289, 411504, 359948, 285773, 59008, 104575, 223454, 441920, 251662, 369840, 482786, 606881, 573969, 460139, 522916, 485032, 357015, 217795, 566584, 333143, 177467, 177594, 123511, 288662, 224398, 232615, 326627, 534392, 522793, 107934, 589497, 187293, 482906, 223360, 219657, 344191, 473633, 280239, 321745, 187742, 585020, 278620, 525886, 453

In [48]:
print(full_mapping[14][3])

[4463, 22871, 3421, 2250, 32519, 14702, 34203, 1457, 27170, 692, 1188, 27084, 29045, 27352, 30966, 1374, 31299, 28613, 1473, 33212, 25224, 1099, 33841, 31347, 28422, 28693, 33008, 34120, 33952, 7710, 14800, 24009, 12071, 27637, 2595, 23106, 20719, 5135, 4013, 30986, 30977, 6171, 12914, 27311, 29003, 1329, 30362, 30003, 25282, 12268, 31816, 4920, 2930, 4927, 6945, 4246, 26936, 31315, 29554, 19090, 26982, 10092, 782, 32124, 33500, 27430, 4633, 32418, 7656, 23334, 17555, 25930, 17987, 29292, 27071, 4518, 2511, 26567, 632, 2633, 3206, 4450, 6987, 7583, 9786, 4192, 29066, 6383, 7057, 15059, 18093, 9689, 18712, 13073, 2426, 28473, 11159, 7595, 25920, 6404, 5859, 4758, 10331, 1862, 10758, 23801, 3884, 10214, 6057, 7017, 7124, 18560, 12273, 17105, 11502, 2860, 8206, 11231, 12824, 10576, 10252, 31772, 11606, 5727, 15479, 10549, 27857, 8797, 11378, 6252, 7254, 178, 4949, 27731, 7032, 3030, 7551, 9637, 6090, 7462, 19923, 3346, 15456, 4668, 2652, 33016, 8080, 10499, 22481, 27057, 11012, 3549, 9310

In [24]:
for i in range(len(full_mapping[0])) :
    list_reads[i].append([full_mapping[0][i], full_mapping[1][i], full_mapping[2][i], full_mapping[3][i], full_mapping[4][i], full_mapping[5][i],
                        full_mapping[6][i], full_mapping[7][i], full_mapping[8][i], full_mapping[9][i], full_mapping[10][i], full_mapping[11][i],
                        full_mapping[12][i], full_mapping[13][i], full_mapping[14][i]])
    
print(list_reads[0])

['TTTCCTTTTT', 'NC_004325.2-100000', 1, [[1, [198275, 346984, 412675, 222197, 222208, 222219, 141536, 278529, 472382, 95770, 370439, 308410, 373485, 593890, 68418, 593802, 423536, 536793, 249822, 201291, 380365, 569951, 361808, 434544, 52645, 389576, 293273, 73765, 444636, 412568, 253527, 436364, 436171, 436187, 436203, 436219, 436235, 436251, 436267, 436283, 436299, 444994, 264242, 225936, 436348, 436315, 292693, 357755, 378191, 357263, 487228, 334026, 412447, 396565, 304229, 226151, 139442]], [1, [384210, 783860, 904071, 872569, 551462, 415938, 645276, 435269, 118278, 421748, 499022, 38263, 134116, 54230, 343857, 814180, 633838, 274860, 51500, 175943, 685739, 378500, 635110, 187210, 308706, 882970, 704344, 841019, 788369, 123222, 400110, 466539, 423507, 82311, 502956, 870641, 13550, 598625, 143916, 96804, 96738, 781436, 923006, 401307, 109746, 406319, 651445, 144591, 774654, 220182, 324104, 82145, 435300, 630711, 458658, 71723, 740236]], [1, [768604, 394636, 319071, 597776, 701969, 6

In [70]:
for el in list_genom :
    print(len(el[0]), el[1])
    
print(3291936/640851)

640851  chromosome: 1
947102  chromosome: 2
1067971  chromosome: 3
1200490  chromosome: 4
1343557  chromosome: 5
1418242  chromosome: 6
1445207  chromosome: 7
1472805  chromosome: 8
1541735  chromosome: 9
1687656 chromosome: 10
2038340 chromosome: 11
2271494 chromosome: 12
2925236 chromosome: 13
3291936 chromosome: 14
34250 D7, apicoplast
5.136819635141398


In [101]:
print(len(list_reads_2[0]))

4


In [92]:
print(len(list_reads_2[0][4][15]))

IndexError: list index out of range

### Création des tables de données

In [102]:
# Table de mapping générale :

table_mapping = np.zeros([2001,18], dtype = object)
table_mapping[0] = ["seq of kmer", "name of sequence", "pos of kmer in seq", "mapping k1", "mapping k2", "mapping k3", "mapping k4",
                   "mapping k5", "mapping k6", "mapping k7", "mapping k8", "mapping k9", "mapping k10", "mapping k11", "mapping k12",
                   "mapping k13", "mapping k14", "mapping D7 api",]

h = 1
for kmer_seq, seq_name, kmer_pos, list_pos in list_reads_2 :
    table_mapping[h][0] = kmer_seq
    table_mapping[h][1] = seq_name
    table_mapping[h][2] = kmer_pos
    for i in range(len(list_pos)) :
        table_mapping[h][i+3] = list_pos[i]
    h += 1
    

"""
for i in range(len(list_reads_2[0][4])) :
    for j in range(len(list_reads_2)) :
        print(list_reads_2[i][4][j])
        print(table_mapping[j+1][i+3])
        table_mapping[j+1][i+3] = list_reads_2[i][4][j]
"""        

print(table_mapping[0:2])

[['seq of kmer' 'name of sequence' 'pos of kmer in seq' 'mapping k1'
  'mapping k2' 'mapping k3' 'mapping k4' 'mapping k5' 'mapping k6'
  'mapping k7' 'mapping k8' 'mapping k9' 'mapping k10' 'mapping k11'
  'mapping k12' 'mapping k13' 'mapping k14' 'mapping D7 api']
 ['TTTCCTTTTT' 'NC_004325.2-100000' 1
  list([198275, 346984, 412675, 222197, 222208, 222219, 141536, 278529, 472382, 95770, 370439, 308410, 373485, 593890, 68418, 593802, 423536, 536793, 249822, 201291, 380365, 569951, 361808, 434544, 52645, 389576, 293273, 73765, 444636, 412568, 253527, 436364, 436171, 436187, 436203, 436219, 436235, 436251, 436267, 436283, 436299, 444994, 264242, 225936, 436348, 436315, 292693, 357755, 378191, 357263, 487228, 334026, 412447, 396565, 304229, 226151, 139442])
  list([384210, 783860, 904071, 872569, 551462, 415938, 645276, 435269, 118278, 421748, 499022, 38263, 134116, 54230, 343857, 814180, 633838, 274860, 51500, 175943, 685739, 378500, 635110, 187210, 308706, 882970, 704344, 841019, 78836

### Fonction d'exportation de données

In [2]:
def export_file(data, filename, prog) :
    
    if prog == "DC3" :
            filename = "DC3_save/" + filename
    elif prog == "Map" :
            filename = "Mapping_save/" + filename
    
    with open(filename, 'w') as f:
        
        # Partie DC3:
        if prog == "DC3" :
            for el in data :
                f.write(str(el) + " ")
                
        if prog == "Map" :
            f.write(str(len(data)) + "\n")
            for el in data :
                f.write(el[0] + "&" + el[1] + "&" + str(el[2]) + "&" + str(el[3]) + "\n" )
        
    
    
def import_file(filename, prog) :
    
    if prog == "DC3" :
            filename = "DC3_save/" + filename
    elif prog == "Map" :
            filename = "Mapping_save/" + filename
    
    with open(filename, 'r') as f:
        
        # Partie DC3:
        if prog == "DC3" :
            text = f.readline()
            data = text.split(" ")[:-1]
            #print(data)
            for i in range(len(data)) :
                data[i] = int(data[i])
            return(data)
        
        elif prog == "Map" :
            n = int(f.readline())
            data = []
            for i in range(n):
                temp = f.readline()
                temp = temp.split("&")
                temp_lis = temp[3][1:-2].split(",") # Car il reste les []
                lis = []
                for j in range(len(temp_lis)) :
                    lis.append(int(temp_lis[j]))
                data.append([temp[0], temp[1], int(temp[2]), lis])
            return data
    

In [9]:
data = DC3_chrom2
filename = "DC3_chrom2.txt"

export_file(data, filename, "DC3")

data_test = import_file(filename, "DC3")

print(data_test == DC3_chrom2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
DC3_chrom1 = import_file("DC3_chrom1.txt", "DC3")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
print(DC3_chrom1 == list_genom[0][2])

True


### Fonction d'importation des données

In [3]:
def genome_import() :
    """
    Fonction qui importe depuis un fichier fasta la séquence du génome dans une liste contenant ["seq", "n°k"]
    """
    list_genom=[]
    for record in SeqIO.parse("GCF_000002765.5_GCA_000002765_genomic.fna","fasta"):
        list_genom.append([str(record.seq).upper(),record.description[-14:]]) # attention le marquage est à changer pour la dernière séquence
    
    return list_genom
    
    
def reads_import() :
    """
    Fonction qui importe depuis un fichier fasta les séquences des reads dans une liste contenant ["seq", "nom"]
    """
    list_reads=[]
    with open("single_Pfal_dat.fq") as in_handle:
        for title, seq, qual in tqdm(FastqGeneralIterator(in_handle), desc = "Import sequence"):
            list_reads.append([str(seq).upper(),title])
            
    return list_reads



def reads_import_cuts(k) :
    """
    Fonction qui importe depuis un fichier fasta les séquences des reads et les coupes en les rangeant
    dans une liste contenant ["seq", "nom", "n°kmer"]
    
    Entrée : k, int : longueur du kmer
    
    Sortie : list_reads : liste de tous les read découpés en kmers
    """
    list_reads=[]
    with open("single_Pfal_dat.fq") as in_handle:
        for title, seq, qual in tqdm(FastqGeneralIterator(in_handle), desc = "Import sequence"):
            i = 1    # Incrémenteur du nombre de k-mer
            while len(seq) >= 1 : # On parcoure toute la séquence
                if len(seq) > k : # Cas où le k-mer est entier
                    list_reads.append([str(seq[0:k]).upper(),title, i])
                    i += 1
                    seq = seq[k:]
                else : # Cas où le dernier k-mer n'est pas entier
                    list_reads.append([str(seq).upper(),title, i])
                    seq = ""
            
    return list_reads

In [4]:
print(reads_import()[0:6])

print(reads_import_cuts(10)[0:12])

Import sequence: 687912it [00:04, 157124.84it/s]


KeyboardInterrupt: 

### Fonction pour le mapping

#### Radix et counting sort

In [4]:
def flatten(arr):
    # voir import
    return ft.reduce(lambda x, y: x + y, arr)

def counting_sort(array,digit,p): 
    """
    Nécessaire pour le radix sort, je te le laisse Erwan
    """
    ##The counting sort is a stable sort used in the radic sort
    ##here the counting sort needs to be adapted to look at only one digit of each number (for radix)
    ##we also added the parameter p to be able to read the triplets (more info below)
    ##1) since we only sort single digits, the max will always be smaller than 10
    ##2) create count index (that will have the cumulative index in the end)
    count_list = [[]for i in range(10)]
    for tpl in array :
        n_uplet = tpl[0]##we take the first element of the tuple, the n-uplet
        num = n_uplet[p] // digit ##we cut off the digits to the right by getting the quotient of the euclidian division
        ##p is the index of the number studied in the triplet
        count_list[num%10].append(tpl)##we cut off the digits to the left by getting the remainder of the euclidian division
    arr_ord = flatten(count_list)

    for i in range(len(array)):##we change the base array to allow the radix sort to loop easily
        array[i] = arr_ord[i]
        
def radix_sort(array,p):
    """
    Prend en entrée un tableau de valeur de valeur et le trie en foncion de la première composante
    selon l'algorithm du radix sort
    
    Entrée : array à trier
             p : nombre de cases à trier p-uplets
    
    Sortie : array trié
    """
    ##Here the radix sort is modified to work with the triplet list sent by the DC3
    ##The code is not flexible enough to compute all characters in the ascii table, but it's enough for the use needed
    ##1) we search for the max in the nuplets
    mx = (max(array)[0])[0]+1
    if p != 3 : ##we take max = 100 because A T C G are all below 100 in ascii code
        for tpl in array:
            n_uplet = tpl[0]
            if n_uplet[-1] > mx :
                mx = n_uplet[-1]+1
    '''##2) to know how many loops we have to do, we will use a variable to represent,
    the digit we are currently in'''
    for i in reversed(range(0,p)):
        digit = 1 ##starts at one for units
        while mx - digit > 0 :##when all the digits are checked, digit will be greater than the maximum
            counting_sort(array,digit,i)
            digit *= 10 ##digit will go to the tens, the hundreds, the thousands...


#### DC3

In [5]:
def DC3(S, P_12_base = []) :
    """
    Create the suffix array from DC3 algorithm, could be recursive if needed
    
    Args:
        S (str): string
        P_12_base : store P1+2 from recursion to map correctly recursivity
    
    Return:
        index_012 : suffix array of S
        order_12 : order of the next recursion to map correctly recursivity
    """
    
    DC3_table = np.zeros((3,len(S) + 3), dtype=int) # Les caractères sentinelles sont déja là !
    """
    Table de DC3 qui contient en chaque ligne : 
    Ligne 0 : indice du caractère
    Ligne 1 : conversion du caractère en nombre
    Ligne 2 : Ordre de l'indice du caractère
    
    """

    for i in range(len(S) + 3) :
            DC3_table[0][i] =  i # On remplace le caractère par son code Ascii
    
    # String conversion : !!!!! à n'executer que lors de la première récursion !!!!!
    if type(S) == str :
        S_l = [*S] # On sépare caractère par caractère : "ATGC" devient ["A","T","G","C"]
        for i in range(len(S_l)) :
            DC3_table[1][i] =  ord(S_l[i]) # On rempli le caractère par son code Ascii dans la table
    else :
         for i in range(len(S)) :
            DC3_table[1][i] =  S[i] # Cas où l'on rentre dans la boucle une deuxième fois ou plus, pas de conversion
    """
    # Cas où la chaine n'est composé que d'une seule lettre : trivial car DC3 = ordre des indices en décroissant
    equal = True
    for i in range(len(S)) :
        if DC3_table[1][0] != DC3_table[1][i] :
            equal = False
            break # On teste si la chaine n'est composé que d'une seule lettre
    if equal == True :
         return [*range(len(S)-1,-1,-1)]
    """
    
    # On crée P0, P1, P2 et P1+P2 :    
    P0 = [*range(0,len(S)+1,3)] 
    P1 = [*range(1,len(S)+1,3)]
    P2 = [*range(2,len(S)+1,3)]
    
    P_12 = P1 + P2
    

    #Obtention des triplets à partir de P1+P2 :
    R_12 = []
    for val in P_12 :
        R_12.append([list(DC3_table[1][val:val+3]), val])
    
    radix_sort(R_12,3) # On trie les triplets

    index_12 = [] # Liste des indexes de R12 trié
    order_count = 1 # Compteur pour remplir l'ordre
    recur = False # Etat de la récursion tourné True si on a des égalités d'ordre
    for j in range(len(R_12)) : # On parcours tous les triplets triés
        index_12.append(R_12[j][1]) # ... pour lui attribuer son index depuis P_12
        DC3_table[2][R_12[j][1]] = order_count # Et on ajoute l'ordre dans la table
        if j < len(R_12)-1 :
            if R_12[j][0] != R_12[j+1][0] : # On teste l'égalité des triplets pour mettre l'ordre
                order_count += 1
            else :
                recur = True # On a égalité, donc on doit relancer l'algorithme à la fin des for
        else :
            order_count += 1

    if recur == True :
        new_S = [] # On crée T' la séquence des orders suivant l'ordre de P12
        for l in P_12 :
            new_S.append(DC3_table[2][l])
        index_012 = DC3(new_S, P_12) # On doit récupérer ces deux paramètres sinon ça marche pas
        index_12 = []
        for ind,val in index_012 :
            DC3_table[2][ind] = val
            index_12.append(ind)
    
    R_0 = [] # On crée la dernière partie à trier
    for val in P0 :
        R_0.append([[int(DC3_table[1][val]), DC3_table[2][val + 1]], val]) # On crée R0 avec son indice
    
    
    radix_sort(R_0,2)
    
    index_0 = [] # Liste des indexes de R0 trié
    for k in range(len(R_0)) : # On parcours tous les doublets triés
       index_0.append(R_0[k][1]) # On récupère l'indice
    
    index_012 = [] # On crée l'index final en ordonant 0 et 1,2
    i_0 = 0
    i_12 = 0
    
    index_12_dict = {}
    for i in range(len(index_12)) :
        index_12_dict[index_12[i]] = i
    
    while (i_0 < len(index_0) and i_12 < len(index_12)) : # On prends tout les éléments : on vide index 0 ou 12
        val_i0 = index_0[i_0]
        val_i12 = index_12[i_12]
        
        if DC3_table[1][val_i0] > DC3_table[1][val_i12] : # Cas où index 12 arrive avant index 0
            index_012.append(index_12[i_12])
            i_12 += 1
        elif DC3_table[1][val_i0] < DC3_table[1][val_i12] : # Cas où index 0 arrive avant index 12
            index_012.append(index_0[i_0])
            i_0 += 1
        else : # Cas d'égalité sur l'indice : si les 2 indexes renvoient le même nombre 
            if index_12[i_12] % 3 == 1 :
                if index_12_dict[val_i0 + 1] > index_12_dict[val_i12 + 1] : # Cas où index 12 au deuxième terme arrive avant index 0 au deuxième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au deuxième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
            else :
                if DC3_table[1][val_i0 + 1] > DC3_table[1][val_i12 + 1] : # On teste dabord cas où index 12 + 1 arrive avant index 0 + 1
                    index_012.append(index_12[i_12])
                    i_12 += 1
                elif DC3_table[1][val_i0 + 1] < DC3_table[1][val_i12 + 1] : # Cas où index 0 + 1 arrive avant index 12 + 1
                    index_012.append(index_0[i_0])
                    i_0 += 1
                elif index_12_dict[val_i0 + 2] > index_12_dict[val_i12 + 2] : # Cas où index 12 au deuxième terme arrive avant index 0 au troisième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au troisième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
    
    index_012.extend(index_12[i_12:]) # Si 1 des deux index est encore plein, on ajoute son contenu
    index_012.extend(index_0[i_0:])

    if int(DC3_table[1][index_012[0]]) == 0 : # On enlève le terme sentinel s'il est présent
        index_012 = index_012[1:]

    if len(P_12_base) > 0 : # Mapping sur recursion -1 si existe
        new_index_012 = []
        for n in range(len(index_012)) :
            new_index_012.append([P_12_base[index_012[n]], n])
        index_012 = new_index_012
   

    return index_012 # Retourne le suffix array si dernière récursion

#### BWT

In [6]:
def BWT(text,suffix_table):
    """
    Compute the BWT from the suffix table

    Args:
        T (str): string
        end_of_string (char): end of string character to append

    Return:
        bwt (str): BWT
    """
    bwt = ""
    sf_tab = suffix_table
    for i in range(len(sf_tab)):
        crt = sf_tab[i]
        bwt += text[crt-1]
    return(bwt)

#### Pattern matching

In [7]:
def pattern_matching_BWT(S,pattern,bwt,index,somme):
    """
    Search a pattern in a String using the BWT

    Args:
        S (str): string
        pattern (str): pattern
        bwt : the bwt of the text (to not compute it each time)

    Return:
        bool: true if the pattern is in the string
        int : position of the first occurence of the pattern in the ordered text
        int : position of the last occurence of the pattern in the ordered text
    """
    pattern_in_S = False
    L = list(bwt)
    lpattern = list(pattern)
    start_string = -1
    end_string = -1
    e = 0
    f = len(L)
    ##init des valeurs utiles pour la substring search
    i = len(lpattern)-1
    X = lpattern[i]
    for tpl in somme :
        if tpl[0]<X:
            e = tpl[1]+1 ##donne place du premier char dans la liste ordonnée
        if tpl[0]==X:
            f = tpl[1]-1 ##donne place du dernier char

    while e < f and i > 0 :
        X = lpattern[i]
        Y = lpattern[i-1]
        suite_impos = True
        r = e
        s = f
        for u in range(r,s+1):
            if(suite_impos==False):##we use the boolean to exit the loop early
                break
            if(L[u]==Y):
                suite_impos= False
                prev = 0
                idx = index[u]
                for tpl in somme :
                    if tpl[0]<Y:
                        prev = tpl[1]
                e = prev + idx[1] -1 ##car l'index commence à 1 et non 0
                start_string = e

        char_found = False
        for u in reversed(range(r,s+1)):
            if(char_found or suite_impos):
                break
            if(L[u]==Y):
                char_found = True
                prev = 0
                idx = index[u]
                for tpl in somme :
                    if tpl[0]<Y:
                        prev = tpl[1]
                f = prev + idx[1] -1 ##car l'index commence à 1 et non 0
                end_string = f
        if suite_impos :## this will stop the loop if no char has been found in the previous one
            break
        i -= 1
    if suite_impos:
        i = 0
        pattern_in_S = False
        return pattern_in_S,start_string,end_string
    ##dans le cas où e = f, il faut vérifier que le reste du substring est bon
    if i > 0 :
        while i > 0 :
            if L[e] != lpattern[i-1]:
                break
            prev = 0
            id=index[e]
            start_string = e
            end_string = e
            for tpl in somme :
                if tpl[0]<L[e]:
                    prev = tpl[1]

            e = prev + id[1]
            i -= 1
    if i < 1 :
        pattern_in_S = True
    return pattern_in_S,start_string,end_string

#### String location

In [8]:
def string_location(text,string,matches,suffix_table):
    '''
    Gives the position of each occurence of the substring in the text

    Args :
        text (string) : the text to search in
        string (string) : the substring to be searched

    Return :
        ???
    '''
    list_occur = []
    #print(suffix_table,"res")

    if matches[0] == False :
        #print("No occurence of the substring was found")
        list_occur.append(-1)
    else :
        for idx in suffix_table[matches[1]:(matches[2]+1)]:
            #print(idx)
            list_occur.append(idx)
            #print(text[idx:idx+len(string)])
    return(list_occur)

def k_positioning(text,pattern,bwt,suffix_table):##permet d'obtenir la liste des positions
    ##initialisation de l'alphabet, de l'index et du compteur total de char
    L = list(bwt)
    alphabet = ['$','A','C','G','T']
    total = []
    index = L+[]
    for lett in alphabet:
        cntr = 0
        for i in range(len(L)):
            if L[i]==lett:
                cntr +=1
                index[i]=(lett,cntr)
        total.append((lett,cntr))##le faire en une liste somme et total
    somme = []
    som=0
    for tpl in total:
        som += tpl[1]
        somme.append((tpl[0],som))
    ##recuperation des positions des premiers et derniers patterns trouvés
    mat = pattern_matching_BWT(text,pattern,bwt,index,somme)
    ##recupération et renvoi des positions de tout les patterns
    return string_location(text,pattern,mat,suffix_table)

In [26]:
sf_tab = list_genom[0][2]
bwt = BWT(list_genom[0][0],sf_tab)
print((k_positioning(list_genom[0][0],list_reads[0][0], bwt,list_genom[0][2])))

[198275, 346984, 412675, 222197, 222208, 222219, 141536, 278529, 472382, 95770, 370439, 308410, 373485, 593890, 68418, 593802, 423536, 536793, 249822, 201291, 380365, 569951, 361808, 434544, 52645, 389576, 293273, 73765, 444636, 412568, 253527, 436364, 436171, 436187, 436203, 436219, 436235, 436251, 436267, 436283, 436299, 444994, 264242, 225936, 436348, 436315, 292693, 357755, 378191, 357263, 487228, 334026, 412447, 396565, 304229, 226151, 139442]


In [20]:
print(list_reads[0][0])

TTTCCTTTTT


In [23]:
print(list_genom[0][2])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Check des kmers

In [34]:
def prepare_reads(data,k):
    """
    give_position of each read
    Args:
        data :List containing the treatment of kmer by mapping :
        (kmer_seq, seq_name, kmer_pos, pos_genom_list)
        kmer_seq : kmer,
        seq_name : read containing the kmer,
        kmer_pos : position of the kmer in the read,
        pos_genom_list : possible positions of the kmer in the genom
        k : length of kmers

    Return:
        ?
    """
    
    """
    output = np.array(["seq_name", "mapping k1", "mapping k2", "mapping k3", "mapping k4",
                      "mapping k5", "mapping k6", "mapping k7", "mapping k8", "mapping k9", 
                      "mapping k10", "mapping k11", "mapping k12",
                      "mapping k13", "mapping k14", "mapping D7 api"], dtype = object)
    
    stock_seq_name= data[1][1]
    start = 0
    for i in range(len(data[0:15])) :
        if stock_seq_name != data[i+1][1] :
            stop = i + 1
            #print(data[start:stop])
            print(output)
            l = 3
            for kmer_seq, seq_name, kmer_pos, pos_genom_list in data[start:stop] :
                output[1][l] = 1
            start = i + 2
            stock_seq_name = data[i+1][1]
    
        
    print(output)
    """
    
    read_pos=[]##creating a list of positions according to the read name with the position in the read
    stock_seq_name= data[0][1] #we take the first name in the list to initialize
    pres_list = []
    i = 0
    temp_pres_list = []
    for kmer_seq, seq_name, kmer_pos, pos_genom_list in tqdm(data) :
        if (seq_name==stock_seq_name):
            read_pos.append(pos_genom_list)
        else :
            for j in range(15) :
                list_for_k = [read_pos[0][j], read_pos[1][j], read_pos[2][j], read_pos[3][j],
                              read_pos[4][j], read_pos[5][j], read_pos[6][j], read_pos[7][j],
                              read_pos[8][j], read_pos[9][j]]
                #print(list_for_k)
                temp_pres_list.append(link_reads(list_for_k,k))
            
            read_pos = [pos_genom_list]
            pres_list.append([stock_seq_name, temp_pres_list])
            stock_seq_name= data[i + 1][1]
            temp_pres_list = []
        i +=1
    
    pres_list.append([stock_seq_name, temp_pres_list])
    
    return pres_list
    
    
def link_reads(l_pos,k):
    """
    Links the kmer of the reads
    Args:
        l_pos : list of tuple : (kmer_pos_read, kmer_pos_gen)
        kmer_pos_read : position of the kmer in the read
        kmer_pos_gen : possible position of the kmer in the genom
        k : length of kmers
    Return:

    """
    valid=False # true if there is a valid position
    val_pos=[] # list of valid starting position of the read in the genom
    for i in l_pos[0][1] :# for each first position,we add the next part to see if it exists
        cur_pos=1
        if(cur_pos==len(l_pos)): # if there is only one position in the list
            if (i!=-1): #if there is only a position -1 it returns False and no position
                valid=True
                val_pos.append(i)

        else :  # if there is more than 1 element iiin the list
            pos_gen=i+k # next position of the kmer of the read in the genom
            while (pos_gen in l_pos[cur_pos][1]) : # we try to find the next kmer in the next list of position of kmer
                cur_pos+=1
                if(cur_pos==len(l_pos)): # when there is the whole read in the genom
                    valid=True
                    val_pos.append(i)# to return the position of the read(s) in the genom
                    break
                else :
                    pos_gen+=k
    return (valid, val_pos)

In [35]:
print(prepare_reads(list_reads[0:80], 10))

100%|████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 9992.98it/s]

[['NC_004325.2-100000', [(False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, [])]], ['NC_004325.2-99999', [(False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, [])]], ['NC_004325.2-99998', [(False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, [])]], ['NC_004325.2-99997', [(False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, [])]], ['NC_004325.2-99996', [(False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (False, []), (Fal

In [26]:
print(list_reads[21])

['CATATATTTG', 'NC_004325.2-99998', 2, [[2, [136164, 271742, 461909, 89646, 471747, 611790, 174794, 329541]], [2, [826948, 189016, 369715, 850398, 192190, 645215, 746133, 389725]], [2, [198871, 764854, 832389, 508169, 218945, 321973, 583467, 889507]], [2, [187807, 237088, 865673, 259749, 859261, 859297, 766155, 423842]], [2, [426744, 983039, 1239979, 1188905, 983358, 1270142, 326762, 512027]], [2, [1039877, 481863, 311639, 507791, 960553, 68424, 1043807, 1412706]], [2, [520521, 536181, 668284, 1214932, 253783, 638754, 1410622, 673860]], [2, [747038, 1117052, 763509, 1136891, 1117076, 40818, 89892, 740296]], [2, [816258, 816297, 816063, 816336, 816141, 816180, 816102, 816375]], [2, [258796, 1420413, 65720, 238841, 1106689, 234196, 996536, 136138]], [2, [618266, 1865871, 1796223, 301122, 294429, 1447771, 145342, 1039986]], [2, [1335747, 1970921, 1379917, 798730, 1605241, 309484, 211142, 1976646]], [2, [1997281, 45843, 147691, 70141, 2481891, 805826, 1748802, 1408226]], [2, [2001330, 2783